<img src="../images/felipepelaquim-9c7CK4BemmA-unsplash.jpg"  style="float: center; margin: 0px; height: 350px; width: 1000px"> 

# Afrobeats influence in Hip Hop

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

import time

In [2]:
#Authentication
client_credentials_manager = SpotifyClientCredentials(client_id = '2101cd224f5948e19c4c782d76744ed3',
                                                      client_secret = '879abdfca432449facc9d8566fb40ab6')

Create spotipy object using SpotifyClientCredentials

In [3]:
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [4]:
# results = sp.playlist_tracks('5ZCzd0nCLqiIX1jwQWfazW')
# tracks = results['items']
    
# while results['next']:
#     results = sp.next(results)
#     tracks.extend(results['items'])

# tracks[4]['track'].keys()

# tracks[4]['track']['uri']

## Get song info

Create a function that takes in a playlist of songs and extracts the songs information and features such as artist name, genre, album name, track_uri, danceability, energy, loudness, instrumentalness etc

In [5]:
def get_track_info(playlist):
    
    #split the playlist_uri is at the end of the playlists url. I'll use the .split method to extract it
    uri = playlist.split("/")[-1].split("?")[0]
    
    #from the spotipy library, use the playlist_tracks() method to extract each track from the playlist uri
    #It comes in a nested dictionary format
    
    results = sp.playlist_tracks(uri)
    tracks = results['items']
    
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    
    
    #create an empty dictionary with the info we want to extract as columns
    info = {
    'track_uri':[],
    'track_name':[],
    'artist_name':[],
    #'artist_info':[],
    'artist_uri':[],
    #'artist_popularity':[],
    #'artist_genre':[],
    'album':[],
    'track_pop':[],
    }
    
    features = {'danceability': [],
     'energy': [],
     'key': [],
     'loudness': [],
     'mode': [],
     'speechiness': [],
     'acousticness': [],
     'instrumentalness': [],
     'liveness': [],
     'valence': [],
     'tempo': [],
     'type': [],
     'id': [],
     'uri': [],
     'track_href': [],
     'analysis_url': [],
     'duration_ms': [],
     'time_signature': []
               }
    
    #using a for loop, get the the info for each song and put it into the empty dictionary
    for track in tracks:
        #URI
        info['track_uri'].append((track["track"]["uri"]).split(':')[2])

        #Track name
        info['track_name'].append(track["track"]["name"])

        #Main Artist
        info['artist_uri'].append((track["track"]["artists"][0]["uri"]).split(':')[2])
        #info['artist_info'].append(sp.artist(track["track"]["artists"][0]["uri"]))

        #Name, popularity, genre
        info['artist_name'].append(track["track"]["artists"][0]["name"])
        #info['artist_popularity'].append(sp.artist(track["track"]["artists"][0]["uri"])["popularity"])
        #info['artist_genre'].append(sp.artist(track["track"]["artists"][0]["uri"])["genres"])

        #Album
        info['album'].append(track["track"]["album"]["name"])

        #Popularity of the track
        info['track_pop'].append(track["track"]["popularity"])
        
        #Transform the info dictionary into a dataframe
        info_df = pd.DataFrame(info)
        
        #loop through the tracks to their features and assign it to the empty dictionary
        track_uri = track["track"]["uri"].split(':')[2] 
        
        try:
            for key,value in (sp.audio_features(track_uri)[0]).items():
                features[key].append(value)
            
        except:
            print(f'failed on track {track["track"]["name"]}')
            continue
        #time.sleep(1)
        
    #Transform the features dictionary into a dataframe
    features_df = pd.DataFrame(features)
    
    #drop features we do not need
    #features_df.drop(columns=['type','id','uri','track_href','analysis_url','duration_ms','time_signature'], inplace=True)   
    
    
    
     
    
    return info_df.join(features_df)
        

#### Create dataframe of users playlist

#### Afrobeats playlist

In [6]:
def raw_data(user_playlist_url, genre):
    user_playlist_info = get_track_info(user_playlist_url)
    #add user genre
    user_playlist_info.loc[:,'genre'] = genre
    return user_playlist_info

In [7]:
afrobeats_playlist_url= "https://open.spotify.com/playlist/5ZCzd0nCLqiIX1jwQWfazW"

In [8]:
afrobeats_df = raw_data(afrobeats_playlist_url, 'afrobeats')

failed on track Lonely


In [13]:
afrobeats_df.head(2)

,track_uri,track_name,artist_name,artist_uri,album,track_pop,danceability,energy,key,loudness,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genre
0,0mDRuZmReEm6DquPLJlcEm,Oleku (feat. Brymo),Ice Prince,1sSt1DqqqFLkPwfrqafVyn,Oleku (feat. Brymo),51,0.544,0.730,1.0,-4.299,...,0.751,162.112,audio_features,0mDRuZmReEm6DquPLJlcEm,spotify:track:0mDRuZmReEm6DquPLJlcEm,https://api.spotify.com/v1/tracks/0mDRuZmReEm6...,https://api.spotify.com/v1/audio-analysis/0mDR...,291364.0,5.0,afrobeats
1,1tvi8tv0eykhNcV1WtaIqO,Move Back,5five,37zb1JQnDV9dRLatrASEj1,Move Back,45,0.793,0.734,11.0,-9.252,...,0.793,125.032,audio_features,1tvi8tv0eykhNcV1WtaIqO,spotify:track:1tvi8tv0eykhNcV1WtaIqO,https://api.spotify.com/v1/tracks/1tvi8tv0eykh...,https://api.spotify.com/v1/audio-analysis/1tvi...,237107.0,4.0,afrobeats


In [14]:
len(afrobeats_df)

1875

In [28]:
afrobeats_df.isna().sum().sum()

0

In [24]:
afrobeats_df[afrobeats_df['energy'].isnull()]

,track_uri,track_name,artist_name,artist_uri,album,track_pop,danceability,energy,key,loudness,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,genre
1874,4bQga8PJWPDIQmecRrqxj9,N'y pense plus,Tayc,7gU9VyFRN3JWPJ5oHOil60,Fleur froide - Second état : la cristallisation,48,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,afrobeats


In [26]:
afrobeats_df.dropna(inplace=True)

In [30]:
afrobeats_df.isnull().sum().sum()

0

#### Exporta afrobeats as a csv

In [31]:
afrobeats_df.to_csv('../data/afrobeats.csv', index = False)

#### Jacks playlist

In [13]:
jacks_playlist_url = 'https://open.spotify.com/playlist/6UlskZAcTPzcGMnQaMnIVm?si=5cbb031d1fdd4064'

In [14]:
jacks_playlist_df = raw_data(jacks_playlist_url, 'jack')

#### Exporta Jack's playlist as a csv

In [18]:
jacks_playlist_df.to_csv('../data/jack.csv', index = False)

#### Ankita playlist

In [15]:
ankita_playlist_url = 'https://open.spotify.com/playlist/6qzbkhrmxdFj5TtaXR0sfI?si=ElY40mMjQ7apOsHTGlQI7A'

In [16]:
ankita_playlist_df = raw_data(ankita_playlist_url, 'ankita')

#### Export Ankita's playlist as a csv

In [19]:
ankita_playlist_df.to_csv('../data/ankita.csv', index = False)

### Get top Afrobeats songs from 2011 to 2022

In [23]:
afrobeats_url_2011 = 'https://open.spotify.com/playlist/7IFO0RwgGq1GTZYH52FtJK'

In [24]:
afrobeats_url_2012 = 'https://open.spotify.com/playlist/0IsKxqhBrseS4cjSvCZI7I'

In [25]:
afrobeats_url_2013 = 'https://open.spotify.com/playlist/2NXEKf7dKn0NNQCuj4Rniz'

In [26]:
afrobeats_url_2014 = 'https://open.spotify.com/playlist/6Vy0hXleg1ojR4mq57w4Fg'

In [27]:
afrobeats_url_2015 = 'https://open.spotify.com/playlist/2fJOQI1m08OKVvjqZyJjCJ'

In [28]:
afrobeats_url_2016 = 'https://open.spotify.com/playlist/5D8Ri2l5XSAsuzZrRVvFqR'

In [29]:
afrobeats_url_2017 = 'https://open.spotify.com/playlist/1yEJSlGQtlpJKTKYEtIed9'

In [30]:
afrobeats_url_2018 = 'https://open.spotify.com/playlist/4KOcyBABIVdTsTGQCK83CP'

In [31]:
afrobeats_url_2019 = 'https://open.spotify.com/playlist/0n79FLMnGldYuQ4yHymtlg'

In [32]:
afrobeats_url_2020 = 'https://open.spotify.com/playlist/7FEPeJeFl364hbm94hUWbo'

In [33]:
afrobeats_url_2021 = 'https://open.spotify.com/playlist/2WiB3m5N21XCO4CP3eKrLP'

In [34]:
afrobeats_url_2022 = 'https://open.spotify.com/playlist/0aUt6gxNcV6B2La64Bm4D5'

#### Function to get create a dataframe of all the songs from 2011 to 2012

In [35]:
afrobeats_2011 = get_track_info(afrobeats_url_2011)

,track_uri,track_name,artist_name,artist_uri,album,track_pop,danceability,energy,key,loudness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,6C6xlP5zYRhYSNI5ORl6Wd,Don't Dull,WizKid,3tVQdUvClmAT7URs9V3rsp,Superstar,52,0.816,0.799,8,-2.173,...,0.0834,0.925,110.008,audio_features,6C6xlP5zYRhYSNI5ORl6Wd,spotify:track:6C6xlP5zYRhYSNI5ORl6Wd,https://api.spotify.com/v1/tracks/6C6xlP5zYRhY...,https://api.spotify.com/v1/audio-analysis/6C6x...,279353,4
1,0mDRuZmReEm6DquPLJlcEm,Oleku (feat. Brymo),Ice Prince,1sSt1DqqqFLkPwfrqafVyn,Oleku (feat. Brymo),51,0.544,0.730,1,-4.299,...,0.1400,0.751,162.112,audio_features,0mDRuZmReEm6DquPLJlcEm,spotify:track:0mDRuZmReEm6DquPLJlcEm,https://api.spotify.com/v1/tracks/0mDRuZmReEm6...,https://api.spotify.com/v1/audio-analysis/0mDR...,291364,5
2,6leKqvSviIydDluSDB7gQz,Kele Kele,Tiwa Savage,1hNaHKp2Za5YdOAG0WnRbc,Once Upon a Time,38,0.712,0.770,7,-3.860,...,0.1030,0.739,109.751,audio_features,6leKqvSviIydDluSDB7gQz,spotify:track:6leKqvSviIydDluSDB7gQz,https://api.spotify.com/v1/tracks/6leKqvSviIyd...,https://api.spotify.com/v1/audio-analysis/6leK...,222693,4
3,0rBd7r99CyWSuziWkgnOMu,Kill Me Shy,D-Cryme,31aYIrTIv5fFcneoyXzq9S,Kill Me Shy - Single,28,0.775,0.806,11,-5.413,...,0.2430,0.825,107.948,audio_features,0rBd7r99CyWSuziWkgnOMu,spotify:track:0rBd7r99CyWSuziWkgnOMu,https://api.spotify.com/v1/tracks/0rBd7r99CyWS...,https://api.spotify.com/v1/audio-analysis/0rBd...,255674,4
4,1a0QPUqSpP67GFJ9eAcvOE,Girl (feat. Wizkid),Bracket,6yd6lB5T20BZjUVkSlwojA,Cupid Stories,42,0.791,0.872,2,-5.940,...,0.0990,0.679,120.017,audio_features,1a0QPUqSpP67GFJ9eAcvOE,spotify:track:1a0QPUqSpP67GFJ9eAcvOE,https://api.spotify.com/v1/tracks/1a0QPUqSpP67...,https://api.spotify.com/v1/audio-analysis/1a0Q...,294960,4
5,4F7gffXvHgQMdojlp3NrNl,One Naira (feat. Waje),M.I. Abaga,0obuUG86gXYcMtRMNKaYKL,Mi2 the Movie,18,0.487,0.588,2,-4.198,...,0.1500,0.499,129.340,audio_features,4F7gffXvHgQMdojlp3NrNl,spotify:track:4F7gffXvHgQMdojlp3NrNl,https://api.spotify.com/v1/tracks/4F7gffXvHgQM...,https://api.spotify.com/v1/audio-analysis/4F7g...,265718,5
6,2iTzNWT7x2NXpMngGZQzPd,U Go Kill Me (feat. El),Sarkodie,01DTVE3KmoPogPZaOvMqO8,Rapperholic,39,0.863,0.886,9,-2.656,...,0.0600,0.781,118.968,audio_features,2iTzNWT7x2NXpMngGZQzPd,spotify:track:2iTzNWT7x2NXpMngGZQzPd,https://api.spotify.com/v1/tracks/2iTzNWT7x2NX...,https://api.spotify.com/v1/audio-analysis/2iTz...,225626,4
7,12n8d0AfedBQysTcN4Tg6W,Obianuju,Duncan Mighty,5N47dnIfPuGH8kIHpf8gZK,Legacy (Ahamefuna),51,0.696,0.693,1,-4.031,...,0.0771,0.935,108.928,audio_features,12n8d0AfedBQysTcN4Tg6W,spotify:track:12n8d0AfedBQysTcN4Tg6W,https://api.spotify.com/v1/tracks/12n8d0AfedBQ...,https://api.spotify.com/v1/audio-analysis/12n8...,221080,4
8,3dx6COHzPCAB1JFFMj58gW,Ajeei (feat. Nana Boroo & Sarkodie),R2Bees,0LFsP7WPfu5inz9a1amcE4,Refuse to Be Broke: Da Revolution 2,33,0.775,0.852,2,-3.414,...,0.3400,0.475,111.845,audio_features,3dx6COHzPCAB1JFFMj58gW,spotify:track:3dx6COHzPCAB1JFFMj58gW,https://api.spotify.com/v1/tracks/3dx6COHzPCAB...,https://api.spotify.com/v1/audio-analysis/3dx6...,220960,4
9,6AIk7Qh81DwTixDSkOFBi9,Azingele,Ruff-N-Smooth,4KSIIR6dY7MPuq5WRWfS56,Life Is Rough & Smooth,26,0.802,0.737,1,-7.640,...,0.0865,0.932,120.018,audio_features,6AIk7Qh81DwTixDSkOFBi9,spotify:track:6AIk7Qh81DwTixDSkOFBi9,https://api.spotify.com/v1/tracks/6AIk7Qh81DwT...,https://api.spotify.com/v1/audio-analysis/6AIk...,284552,4
